In [1]:
import dlib
import sys
import cv2
import time
import numpy as np
from scipy.spatial import distance as dist
from threading import Thread
import playsound
import queue


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2

In [4]:
%%cmd
where python

Microsoft Windows [Version 10.0.22631.4391]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>where python
C:\Users\Yugandran\anaconda3\python.exe
C:\Users\Yugandran\AppData\Local\Microsoft\WindowsApps\python.exe

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>

In [5]:
%%cmd
pip install cmake

Microsoft Windows [Version 10.0.22631.4391]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>pip install cmake

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>

In [6]:
%%cmd
python --version

Microsoft Windows [Version 10.0.22631.4391]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>python --version
Python 3.9.13

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>

In [7]:
%%cmd
pip install "C:\Users\Yugandran\Downloads\dlib-19.22.99-cp39-cp39-win_amd64.whl"

Microsoft Windows [Version 10.0.22631.4391]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>pip install "C:\Users\Yugandran\Downloads\dlib-19.22.99-cp39-cp39-win_amd64.whl"
Processing c:\users\yugandran\downloads\dlib-19.22.99-cp39-cp39-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>

In [8]:
%%cmd
pip install face-recognition

Microsoft Windows [Version 10.0.22631.4391]
(c) Microsoft Corporation. All rights reserved.

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>pip install face-recognition

C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT>

In [9]:
FACE_DOWNSAMPLE_RATIO = 1.5
RESIZE_HEIGHT = 460

thresh = 0.27
modelPath = r"C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT\shape_predictor_68_face_landmarks.dat"
sound_path = r"C:\Users\Yugandran\OneDrive\Desktop\DDD PROJECT\alarm.wav"


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(modelPath)

leftEyeIndex = [36, 37, 38, 39, 40, 41]
rightEyeIndex = [42, 43, 44, 45, 46, 47]

blinkCount = 0
drowsy = 0
state = 0
blinkTime = 0.15  # 150ms
drowsyTime = 1.5  # 1200ms
ALARM_ON = False
GAMMA = 1.5
threadStatusQ = queue.Queue()


In [10]:
invGamma = 1.0 / GAMMA
table = np.array([((i / 255.0) ** invGamma) * 255 for i in range(0, 256)]).astype("uint8")

def gamma_correction(image):
    return cv2.LUT(image, table)

def histogram_equalization(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.equalizeHist(gray)


In [11]:
def soundAlert(path, threadStatusQ):
    while True:
        if not threadStatusQ.empty():
            FINISHED = threadStatusQ.get()
            if FINISHED:
                break
        playsound.playsound(path)


In [12]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)

    # Optional: print debug information
    print(f"EAR: {ear}")  # Debug statement
    
    return ear


In [13]:
def checkEyeStatus(landmarks):
    mask = np.zeros(frame.shape[:2], dtype=np.float32)
    
    hullLeftEye = [(landmarks[i][0], landmarks[i][1]) for i in leftEyeIndex]
    hullRightEye = [(landmarks[i][0], landmarks[i][1]) for i in rightEyeIndex]

    cv2.fillConvexPoly(mask, np.int32(hullLeftEye), 255)
    cv2.fillConvexPoly(mask, np.int32(hullRightEye), 255)

    leftEAR = eye_aspect_ratio(hullLeftEye)
    rightEAR = eye_aspect_ratio(hullRightEye)

    ear = (leftEAR + rightEAR) / 2.0

    # Optional: print debug information
    print(f"Left EAR: {leftEAR}, Right EAR: {rightEAR}, Combined EAR: {ear}")  # Debug statement

    return 1 if ear >= thresh else 0  # 1 -> Open, 0 -> closed



In [14]:
def getLandmarks(im):
    imSmall = cv2.resize(im, None, 
                         fx=1.0 / FACE_DOWNSAMPLE_RATIO, 
                         fy=1.0 / FACE_DOWNSAMPLE_RATIO, 
                         interpolation=cv2.INTER_LINEAR)

    rects = detector(imSmall, 0)
    if len(rects) == 0:
        return 0

    newRect = dlib.rectangle(int(rects[0].left() * FACE_DOWNSAMPLE_RATIO),
                             int(rects[0].top() * FACE_DOWNSAMPLE_RATIO),
                             int(rects[0].right() * FACE_DOWNSAMPLE_RATIO),
                             int(rects[0].bottom() * FACE_DOWNSAMPLE_RATIO))

    points = [(p.x, p.y) for p in predictor(im, newRect).parts()]
    return points


In [15]:
def getLandmarks(im):
    imSmall = cv2.resize(im, None, 
                         fx=1.0 / FACE_DOWNSAMPLE_RATIO, 
                         fy=1.0 / FACE_DOWNSAMPLE_RATIO, 
                         interpolation=cv2.INTER_LINEAR)

    rects = detector(imSmall, 0)
    if len(rects) == 0:
        return 0

    newRect = dlib.rectangle(int(rects[0].left() * FACE_DOWNSAMPLE_RATIO),
                             int(rects[0].top() * FACE_DOWNSAMPLE_RATIO),
                             int(rects[0].right() * FACE_DOWNSAMPLE_RATIO),
                             int(rects[0].bottom() * FACE_DOWNSAMPLE_RATIO))

    points = [(p.x, p.y) for p in predictor(im, newRect).parts()]
    return points


In [16]:
capture = cv2.VideoCapture(0)

for _ in range(10):
    ret, frame = capture.read()

totalTime = 0.0
validFrames = 0
dummyFrames = 100

print("Calibration in Progress!")
while validFrames < dummyFrames:
    validFrames += 1
    t = time.time()
    ret, frame = capture.read()
    height, width = frame.shape[:2]
    IMAGE_RESIZE = np.float32(height) / RESIZE_HEIGHT
    frame = cv2.resize(frame, None, fx=1 / IMAGE_RESIZE, fy=1 / IMAGE_RESIZE, interpolation=cv2.INTER_LINEAR)
    
    adjusted = histogram_equalization(frame)
    landmarks = getLandmarks(adjusted)
    timeLandmarks = time.time() - t

    if landmarks == 0:
        validFrames -= 1
        cv2.putText(frame, "Unable to detect face, Please check proper lighting", (10, 30), 
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow("Blink Detection Demo", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            sys.exit()
    else:
        totalTime += timeLandmarks

print("Calibration Complete!")
spf = totalTime / dummyFrames
print(f"Current SPF (seconds per frame) is {spf * 1000:.2f} ms")

drowsyLimit = drowsyTime / spf
falseBlinkLimit = blinkTime / spf
print(f"drowsy limit: {drowsyLimit}, false blink limit: {falseBlinkLimit}")


Calibration in Progress!
Calibration Complete!
Current SPF (seconds per frame) is 52.13 ms
drowsy limit: 28.773275748877587, false blink limit: 2.8773275748877585


In [ ]:
if __name__ == "__main__":
    vid_writer = cv2.VideoWriter('output-low-light-2.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1], frame.shape[0]))
    
    while True:
        try:
            t = time.time()
            ret, frame = capture.read()
            height, width = frame.shape[:2]
            IMAGE_RESIZE = np.float32(height) / RESIZE_HEIGHT
            frame = cv2.resize(frame, None, fx=1 / IMAGE_RESIZE, fy=1 / IMAGE_RESIZE, interpolation=cv2.INTER_LINEAR)

            adjusted = histogram_equalization(frame)
            landmarks = getLandmarks(adjusted)

            if landmarks == 0:
                cv2.putText(frame, "Unable to detect face, Please check proper lighting", (10, 30), 
                            cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow("Blink Detection Demo", frame)
                if cv2.waitKey(1) & 0xFF == 27:
                    break
                continue

            eyeStatus = checkEyeStatus(landmarks)
            checkBlinkStatus(eyeStatus)

            for i in range(0, len(leftEyeIndex)):
                cv2.circle(frame, (landmarks[leftEyeIndex[i]][0], landmarks[leftEyeIndex[i]][1]), 1, (0, 0, 255), -1, lineType=cv2.LINE_AA)

            for i in range(0, len(rightEyeIndex)):
                cv2.circle(frame, (landmarks[rightEyeIndex[i]][0], landmarks[rightEyeIndex[i]][1]), 1, (0, 0, 255), -1, lineType=cv2.LINE_AA)

            if drowsy:
                cv2.putText(frame, "! ! ! DROWSINESS ALERT ! ! !", (70, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                if not ALARM_ON:
                    ALARM_ON = True
                    threadStatusQ.put(not ALARM_ON)
                    thread = Thread(target=soundAlert, args=(sound_path, threadStatusQ,))
                    thread.setDaemon(True)
                    thread.start()
            else:
                cv2.putText(frame, f"Blinks: {blinkCount}", (460, 80), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
                ALARM_ON = False

            cv2.imshow("Blink Detection Demo", frame)
            vid_writer.write(frame)

            if cv2.waitKey(1) & 0xFF == 27:
                break

        except Exception as e:
            print(f"An error occurred: {e}")


EAR: 0.2817240929131168
EAR: 0.3048553804248462
Left EAR: 0.2817240929131168, Right EAR: 0.3048553804248462, Combined EAR: 0.2932897366689815
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.30936160237962834
EAR: 0.3048553804248462
Left EAR: 0.30936160237962834, Right EAR: 0.3048553804248462, Combined EAR: 0.3071084914022373
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.30936160237962834
EAR: 0.3048553804248462
Left EAR: 0.30936160237962834, Right EAR: 0.3048553804248462, Combined EAR: 0.3071084914022373
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.30936160237962834
EAR: 0.3048553804248462
Left EAR: 0.30936160237962834, Right EAR: 0.3048553804248462, Combined EAR: 0.3071084914022373
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2923526731023431
EAR: 0.3241202224214485
Left EAR: 0.2923526731023431, Right EAR: 0.3241202224214485, Combined EAR: 0.3082364477618958
An error occurred: name 'checkBlinkStatus' is not de

EAR: 0.2848778190674648
EAR: 0.2817240929131168
Left EAR: 0.2848778190674648, Right EAR: 0.2817240929131168, Combined EAR: 0.2833009559902908
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2848778190674648
EAR: 0.2817240929131168
Left EAR: 0.2848778190674648, Right EAR: 0.2817240929131168, Combined EAR: 0.2833009559902908
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3048553804248462
EAR: 0.31622776601683794
Left EAR: 0.3048553804248462, Right EAR: 0.31622776601683794, Combined EAR: 0.3105415732208421
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3128430032178382
EAR: 0.3028912664076913
Left EAR: 0.3128430032178382, Right EAR: 0.3028912664076913, Combined EAR: 0.30786713481276473
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3128430032178382
EAR: 0.3028912664076913
Left EAR: 0.3128430032178382, Right EAR: 0.3028912664076913, Combined EAR: 0.30786713481276473
An error occurred: name 'checkBlinkStatus' is not defi

EAR: 0.2923526731023431
EAR: 0.3028912664076913
Left EAR: 0.2923526731023431, Right EAR: 0.3028912664076913, Combined EAR: 0.2976219697550172
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2923526731023431
EAR: 0.3028912664076913
Left EAR: 0.2923526731023431, Right EAR: 0.3028912664076913, Combined EAR: 0.2976219697550172
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2835509473035504
EAR: 0.2835509473035504
Left EAR: 0.2835509473035504, Right EAR: 0.2835509473035504, Combined EAR: 0.2835509473035504
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2923526731023431
EAR: 0.3028912664076913
Left EAR: 0.2923526731023431, Right EAR: 0.3028912664076913, Combined EAR: 0.2976219697550172
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2923526731023431
EAR: 0.3028912664076913
Left EAR: 0.2923526731023431, Right EAR: 0.3028912664076913, Combined EAR: 0.2976219697550172
An error occurred: name 'checkBlinkStatus' is not defined


EAR: 0.3262219966972523
EAR: 0.3109879245816556
Left EAR: 0.3262219966972523, Right EAR: 0.3109879245816556, Combined EAR: 0.318604960639454
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3128430032178382
EAR: 0.33135675230376294
Left EAR: 0.3128430032178382, Right EAR: 0.33135675230376294, Combined EAR: 0.32209987776080057
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3128430032178382
EAR: 0.33135675230376294
Left EAR: 0.3128430032178382, Right EAR: 0.33135675230376294, Combined EAR: 0.32209987776080057
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2923526731023431
EAR: 0.3241202224214485
Left EAR: 0.2923526731023431, Right EAR: 0.3241202224214485, Combined EAR: 0.3082364477618958
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2923526731023431
EAR: 0.3241202224214485
Left EAR: 0.2923526731023431, Right EAR: 0.3241202224214485, Combined EAR: 0.3082364477618958
An error occurred: name 'checkBlinkStatus' is not def

EAR: 0.2835509473035504
EAR: 0.2835509473035504
Left EAR: 0.2835509473035504, Right EAR: 0.2835509473035504, Combined EAR: 0.2835509473035504
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.29584971897989776
EAR: 0.31846487764924103
Left EAR: 0.29584971897989776, Right EAR: 0.31846487764924103, Combined EAR: 0.3071572983145694
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2634736922190852
EAR: 0.28176687428019115
Left EAR: 0.2634736922190852, Right EAR: 0.28176687428019115, Combined EAR: 0.27262028324963816
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2977229608361267
EAR: 0.2817240929131168
Left EAR: 0.2977229608361267, Right EAR: 0.2817240929131168, Combined EAR: 0.2897235268746218
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2977229608361267
EAR: 0.2817240929131168
Left EAR: 0.2977229608361267, Right EAR: 0.2817240929131168, Combined EAR: 0.2897235268746218
An error occurred: name 'checkBlinkStatus' is not d

...

EAR: 0.32943962399405374
EAR: 0.31622776601683794
Left EAR: 0.32943962399405374, Right EAR: 0.31622776601683794, Combined EAR: 0.32283369500544584
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.32943962399405374
EAR: 0.31622776601683794
Left EAR: 0.32943962399405374, Right EAR: 0.31622776601683794, Combined EAR: 0.32283369500544584
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2944406204482613
EAR: 0.3023359340269785
Left EAR: 0.2944406204482613, Right EAR: 0.3023359340269785, Combined EAR: 0.2983882772376199
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2944406204482613
EAR: 0.3023359340269785
Left EAR: 0.2944406204482613, Right EAR: 0.3023359340269785, Combined EAR: 0.2983882772376199
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2575139850557298
EAR: 0.27116715405633474
Left EAR: 0.2575139850557298, Right EAR: 0.27116715405633474, Combined EAR: 0.26434056955603225
An error occurred: name 'checkBlinkStatus' is

EAR: 0.30493901829390213
EAR: 0.29288198698369483
Left EAR: 0.30493901829390213, Right EAR: 0.29288198698369483, Combined EAR: 0.2989105026387985
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.30493901829390213
EAR: 0.29288198698369483
Left EAR: 0.30493901829390213, Right EAR: 0.29288198698369483, Combined EAR: 0.2989105026387985
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.31506776622639476
EAR: 0.31983637960287664
Left EAR: 0.31506776622639476, Right EAR: 0.31983637960287664, Combined EAR: 0.31745207291463573
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3144288124837404
EAR: 0.30787404357342735
Left EAR: 0.3144288124837404, Right EAR: 0.30787404357342735, Combined EAR: 0.31115142802858387
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3144288124837404
EAR: 0.30787404357342735
Left EAR: 0.3144288124837404, Right EAR: 0.30787404357342735, Combined EAR: 0.31115142802858387
An error occurred: name 'checkBlinkStat

EAR: 0.2944406204482613
EAR: 0.2863487550540785
Left EAR: 0.2944406204482613, Right EAR: 0.2863487550540785, Combined EAR: 0.2903946877511699
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2944406204482613
EAR: 0.2863487550540785
Left EAR: 0.2944406204482613, Right EAR: 0.2863487550540785, Combined EAR: 0.2903946877511699
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.32360628863314517
EAR: 0.31104256586080126
Left EAR: 0.32360628863314517, Right EAR: 0.31104256586080126, Combined EAR: 0.3173244272469732
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3243029660565926
EAR: 0.31484548027129766
Left EAR: 0.3243029660565926, Right EAR: 0.31484548027129766, Combined EAR: 0.3195742231639451
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3243029660565926
EAR: 0.31484548027129766
Left EAR: 0.3243029660565926, Right EAR: 0.31484548027129766, Combined EAR: 0.3195742231639451
An error occurred: name 'checkBlinkStatus' is not 

EAR: 0.3057557686100365
EAR: 0.30101437185919644
Left EAR: 0.3057557686100365, Right EAR: 0.30101437185919644, Combined EAR: 0.30338507023461647
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.3057557686100365
EAR: 0.30101437185919644
Left EAR: 0.3057557686100365, Right EAR: 0.30101437185919644, Combined EAR: 0.30338507023461647
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2860902828111481
EAR: 0.30101437185919644
Left EAR: 0.2860902828111481, Right EAR: 0.30101437185919644, Combined EAR: 0.2935523273351723
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2860902828111481
EAR: 0.30101437185919644
Left EAR: 0.2860902828111481, Right EAR: 0.30101437185919644, Combined EAR: 0.2935523273351723
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.2860902828111481
EAR: 0.2871832634470952
Left EAR: 0.2860902828111481, Right EAR: 0.2871832634470952, Combined EAR: 0.28663677312912167
An error occurred: name 'checkBlinkStatus' is n

EAR: 0.33275762637414663
EAR: 0.3425889211774961
Left EAR: 0.33275762637414663, Right EAR: 0.3425889211774961, Combined EAR: 0.33767327377582135
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.33275762637414663
EAR: 0.3425889211774961
Left EAR: 0.33275762637414663, Right EAR: 0.3425889211774961, Combined EAR: 0.33767327377582135
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.318064806566743
EAR: 0.3270357717913192
Left EAR: 0.318064806566743, Right EAR: 0.3270357717913192, Combined EAR: 0.3225502891790311
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.318064806566743
EAR: 0.3270357717913192
Left EAR: 0.318064806566743, Right EAR: 0.3270357717913192, Combined EAR: 0.3225502891790311
An error occurred: name 'checkBlinkStatus' is not defined
EAR: 0.318064806566743
EAR: 0.3270357717913192
Left EAR: 0.318064806566743, Right EAR: 0.3270357717913192, Combined EAR: 0.3225502891790311
An error occurred: name 'checkBlinkStatus' is not defined


In [ ]:
capture.release()
vid_writer.release()
cv2.destroyAllWindows()
